<a href="https://colab.research.google.com/github/fatemafaria142/From-Text-to-Code-Generation-Pipeline-Incorporating-Various-LLMs/blob/main/Code_Generation_using_Mistral_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.8 MB/s eta 0:00:00


### **Dataset Link:** https://huggingface.co/datasets/sahil2801/CodeAlpaca-20k

In [2]:
from datasets import load_dataset

instruct_tune_dataset = load_dataset("sahil2801/CodeAlpaca-20k")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

### **Dataset structure**

* The dataset contains four different columns. We are only interested in the columns `instruction` and `response`. There are 8 different possible source value in the `category` column. We are only interested in one of them.

In [4]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 20022
    })
})

In [10]:
# Display information for 5 data points from the 'train' split
num_samples_to_show = 5
for i in range(num_samples_to_show):
    data = instruct_tune_dataset['train'][i]
    print(f"Data Point {i + 1}:")
    print("Instruction:", data['instruction'])
    print("Input:", data['input'])
    print("Output:", data['output'])
    print("\n-----------------------------\n")


Data Point 1:
Instruction: Create an array of length 5 which contains all even numbers between 1 and 10.
Input: 
Output: arr = [2, 4, 6, 8, 10]

-----------------------------

Data Point 2:
Instruction: Formulate an equation to calculate the height of a triangle given the angle, side lengths and opposite side length.
Input: 
Output: Height of triangle = opposite side length * sin (angle) / side length

-----------------------------

Data Point 3:
Instruction: Write a replace method for a string class which replaces the given string with a given set of characters.
Input: string = "Hello World!"
replace_with = "Greetings!"
Output: def replace(self, replace_with):
    new_string = ""
    for char in self:
        if char == " ":
            new_string += replace_with
        else:
            new_string += char
    return new_string

-----------------------------

Data Point 4:
Instruction: Create an array of length 15 containing numbers divisible by 3 up to 45.
Input: 
Output: arr = [3, 

### **We will use just a small subset of the data for this training example**

In [7]:
instruct_tune_dataset["train"] = instruct_tune_dataset["train"].select(range(1500))
instruct_tune_dataset["test"] = instruct_tune_dataset["train"].select(range(200))

In [8]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 200
    })
})

In [11]:
def create_prompt(sample):
  """
  Update the prompt template:
  Combine both the prompt and input into a single column.

  """
  bos_token = "<s>"
  original_system_message = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
  system_message = "Use the provided input to create an instruction that could have been used to generate the response with an LLM."
  response = sample["output"].replace(original_system_message, "").replace("\n\n### Instruction\n", "").replace("\n### Response\n", "").strip()
  input = sample["instruction"]
  eos_token = "</s>"

  full_prompt = ""
  full_prompt += bos_token
  full_prompt += "### Instruction:"
  full_prompt += "\n" + system_message
  full_prompt += "\n\n### Input:"
  full_prompt += "\n" + input
  full_prompt += "\n\n### Response:"
  full_prompt += "\n" + response
  full_prompt += eos_token

  return full_prompt

In [12]:
create_prompt(instruct_tune_dataset["train"][0])

'<s>### Instruction:\nUse the provided input to create an instruction that could have been used to generate the response with an LLM.\n\n### Input:\nCreate an array of length 5 which contains all even numbers between 1 and 10.\n\n### Response:\narr = [2, 4, 6, 8, 10]</s>'

### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1

In [14]:
mode_id = "mistralai/Mistral-7B-Instruct-v0.1"

In [15]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [16]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

### **Let's example how well the model does at this task currently:**

In [17]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [21]:
prompt="### Instruction:\nUse the provided input to create an instruction that could have been used to generate the response with an LLM.### Input:\nCreate an array of length 5 which contains all even numbers between 1 and 10.\n\n### Response:"

In [22]:
generate_response(prompt, model)

'<s> \nHere is one way to create an array of length 5 that contains all even numbers between 1 and 10 using Python:\n\n```python\nmy_array = [2, 4, 6, 8, 10]\n```</s>'

### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [23]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [24]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [25]:
from transformers import TrainingArguments
output_model= "mistral_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=50,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [26]:
from trl import SFTTrainer

max_seq_length = 1024

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt, # this will aplly the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=instruct_tune_dataset["train"],
  eval_dataset=instruct_tune_dataset["test"]
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:304: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [27]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,0.853700
20,0.631500
30,0.548300
40,0.532700
50,0.538600


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=50, training_loss=0.6209576225280762, metrics={'train_runtime': 789.2866, 'train_samples_per_second': 0.253, 'train_steps_per_second': 0.063, 'total_flos': 8741766719078400.0, 'train_loss': 0.6209576225280762, 'epoch': 1.1})

### **Save the model**

In [28]:
trainer.save_model("mistral_instruct_generation")

In [29]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [30]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

In [35]:
# Example usage
prompt = "### Instruction:\nUse the provided input to create an instruction that could have been used to generate the response with an LLM."
prompt += "\n### Input:\nWrite code that removes spaces from a given string."
prompt += "\n\n### Response:"
response = generate_response(prompt, model)

# Print the response with formatted output
print(response)

<s> ### Instruction:
Use the provided input to create an instruction that could have been used to generate the response with an LLM.
### Input:
Write code that removes spaces from a given string.

### Response:
Here's a Python code that removes spaces from a given string using the `join()` method:

```
def remove_spaces(string):
    # split the string into a list of characters
    characters = list(string)
    # remove all spaces from the list
    characters.remove(' ')
    # join the remaining characters back into a single string
    return ''.join(characters)
```</s>


In [36]:
# Example usage
prompt = "### Instruction:\nUse the provided input to create an instruction that could have been used to generate the response with an LLM."
prompt += "\n### Input:\nWrite a function to generate the nth Fibonacci number."
prompt += "\n\n### Response:"
response = generate_response(prompt, model)

# Print the response with formatted output
print(response)

<s> ### Instruction:
Use the provided input to create an instruction that could have been used to generate the response with an LLM.
### Input:
Write a function to generate the nth Fibonacci number.

### Response:
Here is an example Python function that can generate the nth Fibonacci number:

```python
def fibonacci(n):
  if n < 0:
    return "Invalid input"
  elif n == 0 or n == 1:
    return n
  else:
    # initialize variables
    a = 0
    b = 1
    # check if n is greater than 1
    if n > 1:
      # recursively call the function with n-1 and n-2 as parameters until n is 0 or 1 
      return (a + b) * fibonacci(n-1)
```</s>


In [37]:
# Example usage
prompt = "### Instruction:\nUse the provided input to create an instruction that could have been used to generate the response with an LLM."
prompt += "\n### Input:\nWrite a replace method for a string class which replaces the given string with a given set of characters."
prompt += "\n\n### Response:"
response = generate_response(prompt, model)

# Print the response with formatted output
print(response)

<s> ### Instruction:
Use the provided input to create an instruction that could have been used to generate the response with an LLM.
### Input:
Write a replace method for a string class which replaces the given string with a given set of characters.

### Response:
```python
class StringClass:
    def __init__(self, str):
        self.str = str

    def replace(self, char, new_str):
        for i in range(len(self.str)):
            if self.str[i] == char:
                self.str = self.str[:i] + new_str[i:]
                break
        return self.str
```</s>
